## Autors Gunārs Ābeltiņš, ga22008

### Pamats Jupyter Notebook veidošanai.

1) Koda izpildi kontrolē viens mainīgais - slēdzis  
2) Atkarībā no slēdža stāvokļa, tiek izpildīts tikai modeļa apmācībai vai tikai modeļa pārbaudei nepieciešamais kods  
3) Tiek ielādēti tikai tie moduļi un dati, kas nepieciešami konkrētās darbības izpildei  
4) Ja, pārbaudot kodu, nepieciešams darīt vairāk nekā nomainīt slēdzi un norādīt testa datu kopas ielādes URL, darbs nav korekti pabeigts, notestēts, utt  
5) Pabeigta darba kodam jāstrādā bez aizķeršanās, palaižot to ar Run All  
6) Pēc iespējas visu darbu darīt ar klasēm un funkcijām

### Kontroles bloks
Definē slēdzi un satur saites uz datiem un trenēto modeli

In [1]:
# True - ielādē treniņa datu kopu un uz tās apmāca jaunu modeli, kuru saglabā
# False - ielādē jau apmācītu modeli un atsevišķu datu kopu, uz kuras modeli pārbaudīt
TRAIN = False

# Norādam apmācīta modeļa ielādes URL. Nomainīt.
MODEL_URL = "https://dl.dropbox.com/scl/fi/r6iy31winl7u79bacak93/iris_model-1.pth?rlkey=ca646luvn4mne1cbvp2osdqpa&st=dx1to0em&dl=1"
# Minamāla modeļa URL.
MINIMAL_MODEL_URL = "https://dl.dropbox.com/scl/fi/szsngp0m39erbwmby5e05/minimal_iris_model.pth?rlkey=92nkmamp275e9aub4nqxz2z1x&st=ywosd2ha&dl=1"
# Norādam treniņa datu kopas ielādes URL. Nomainīt.
TRAIN_DATA_URL = 'https://dl.dropbox.com/scl/fi/jcb4bv8g1g2s69a93hmae/Iris_train.csv?rlkey=shqgajvrhpxsw04ptb4nuvgle&dl=1' # <saite uz treniņa datu kopu gdrive, dropbox vai citā serverī>
# Norādam testa datu kopas ielādes URL. Testējot kodu, pagaidām ir validācijas datu kopa, taču pārbaudot darbu pasniedzējs nomainīs uz nodalītu vērtēšanas kopu
TEST_DATA_URL = 'https://dl.dropbox.com/scl/fi/xmsnbuxafq70joj56bcc6/Iris_validate.csv?rlkey=idzjb2shqpypkv23aayajqjsh&dl=1' # <saite uz testa datu kopu gdrive, dropbox vai citā serverī>
# Dropbox gadījumā - padoms tiešās saites iegūšanai https://www.dropboxforum.com/discussions/101001014/public-links-to-raw-files/110391 

### Moduļu importēšana
Importē nepieciešamos moduļus.

In [2]:
# os modulis nepieciešams darbam ar failiem
import os
# torch modulis nodrošina MI apmācībai un lietošanai nepieciešamās funkcijas
import torch
from torch import nn
# numpy modulis nodrošina dažādas matemātiskās funkcijas
import numpy as np
# Pandas modulis tabulveida datu apstrādei
import pandas as pd
# utt.
import torch.utils.data as data
import torch.nn.functional as F

import matplotlib.pyplot as plt

import requests
import io

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# GPU operācijām ir atsevišķs random seed, kas atšķiras no 'standarta' random seed uz CPU vei
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

# Daļa GPU operāciju var būt ar nedeterminētu secību, kas ļauj apstrādi veikt mazliet ātrāk
# Ar šiem uzstādījumiem to noņem, lai ir reproducējami tie paši rezultāti
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Klašu un funkciju definēšana
Ieteicams visu definēt koda sākumā.  
Komentējiet pašas funkcijas mērķi un galvenās darbības tās iekšienē.  
Ja nepieciešams, definējiet papildus klases un funkcijas.


In [3]:
# Definē klasi DataSet, kas saturēs datus neironu tīklam saprotamā formātā

class IrisDataset(data.Dataset):
    def __init__(self, df):
        # Convert feature columns to float
        self.features = torch.tensor(df.iloc[:, 0:4].astype(float).values, dtype=torch.float32)
        
        # Fixed class mapping regardless of data distribution
        self.class_map = {
            'Iris-setosa': 0,
            'Iris-versicolor': 1, 
            'Iris-virginica': 2
        }
        
        # Map labels using the fixed class map
        self.labels = torch.tensor([self.class_map[label] for label in df.iloc[:, 4].values], dtype=torch.long)
        
    def __len__(self):
        return len(self.features)
        
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Definē klasi Model - pašu neironu tīklu

class IrisModel(nn.Module):
  # Ieejas slānis(4 ziedu parametri)
  # Paslēpti slānis h1 un h2 (kuru vērtība ir neironu skaits)
  # Izvadē būs 3 iespējamie Īrisu ziedu veidi

  def __init__(self, in_features=4, h1=10, h2=11, out_features=3):
    super().__init__()
    self.fc1 = nn.Linear(in_features, h1)
    self.fc2 = nn.Linear(h1,h2)
    self.out = nn.Linear(h2,out_features)

#Izmantojam relu aktivācijas funkciju
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x

class MinimalLinearIrisModel(nn.Module):
    def __init__(self, in_features=4, out_features=3):
        super().__init__()
        self.linear = nn.Linear(in_features,out_features)

    def forward(self, x):
        return self.linear(x)

def trainModel(Model, Dataset, epochs, learning_rate, save_path, train_split=0.8):
    # Create full dataset
    full_dataset = Dataset(iris_dataframe)
    
    # Calculate split sizes
    dataset_size = len(full_dataset)
    train_size = int(train_split * dataset_size)
    val_size = dataset_size - train_size
    
    # Split dataset
    train_dataset, val_dataset = torch.utils.data.random_split(
        full_dataset, [train_size, val_size]
    )
    
    # Create data loaders
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=20, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=val_size, shuffle=False)
    
    # Initialize model
    model = Model()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss_function = nn.CrossEntropyLoss()
    
    # Track metrics
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    
    best_val_accuracy = 0
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_epoch_loss = 0
        train_correct = 0
        train_total = 0
        
        for data_inputs, data_labels in train_loader:
            # Forward pass
            outputs = model(data_inputs)
            loss = loss_function(outputs, data_labels)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Track metrics
            train_epoch_loss += loss.item()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            train_total += data_labels.size(0)
            train_correct += (predicted == data_labels).sum().item()
        
        # Calculate average training metrics
        avg_train_loss = train_epoch_loss / len(train_loader)
        train_accuracy = 100 * train_correct / train_total
        train_losses.append(avg_train_loss)
        train_accuracies.append(train_accuracy)
        
        # Validation phase
        model.eval()
        val_epoch_loss = 0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for data_inputs, data_labels in val_loader:
                # Forward pass
                outputs = model(data_inputs)
                loss = loss_function(outputs, data_labels)
                
                # Track metrics
                val_epoch_loss += loss.item()
                
                # Calculate accuracy
                _, predicted = torch.max(outputs.data, 1)
                val_total += data_labels.size(0)
                val_correct += (predicted == data_labels).sum().item()
        
        # Calculate average validation metrics
        avg_val_loss = val_epoch_loss / len(val_loader)
        val_accuracy = 100 * val_correct / val_total
        val_losses.append(avg_val_loss)
        val_accuracies.append(val_accuracy)
        
        # Save best model
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), save_path)
            
        # Print progress
        if (epoch + 1) % 10 == 0:
            print(f"Epoch: {epoch+1}/{epochs}")
            print(f"  Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}%")
            print(f"  Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")
    
    # Plot loss and accuracy curves
    epochs_range = range(1, epochs + 1)
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot losses
    ax1.plot(epochs_range, train_losses, label='Training Loss', marker='o', linestyle='-', markersize=3)
    ax1.plot(epochs_range, val_losses, label='Validation Loss', marker='s', linestyle='-', markersize=3)
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.set_title('Training and Validation Loss')
    ax1.legend()
    ax1.grid(True, linestyle='--', alpha=0.7)
    
    # Plot accuracies
    ax2.plot(epochs_range, train_accuracies, label='Training Accuracy', marker='o', linestyle='-', markersize=3)
    ax2.plot(epochs_range, val_accuracies, label='Validation Accuracy', marker='s', linestyle='-', markersize=3)
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy (%)')
    ax2.set_title('Training and Validation Accuracy')
    ax2.legend()
    ax2.grid(True, linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Best validation accuracy: {best_val_accuracy:.2f}%")
    print(f"Model saved to {save_path}")

# Definē funkciju train, kuru izsaucot, modelis tiek trenēts ar treniņdatu kopu
def train():
    print("Training model...")

    trainModel(IrisModel, IrisDataset, 200, 0.01, "iris_model.pth")

    print("Training minimal model...")

    trainModel(MinimalLinearIrisModel, IrisDataset, 200, 0.01, "minimal_iris_model.pth")

    print("Training complete.")

def testModel(ModelClass, model_path):
    print(f"Testing {ModelClass.__name__} model...")

    # Load test dataset
    df_test = pd.read_csv(TEST_DATA_URL)
    test_dataset = IrisDataset(df_test)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

    
    class_map = test_dataset.class_map

    # Create reverse class map
    reverse_class_map = {v: k for k, v in class_map.items()}

    # Load trained model
    model = ModelClass()
    model.load_state_dict(torch.load(model_path, weights_only=True))
    model.eval()  # Set model to evaluation mode

    correct = 0
    total = len(test_dataset)

    with torch.no_grad():
        for idx, (inputs, true_label) in enumerate(test_loader):
            outputs = model(inputs)
            predicted_idx = torch.argmax(outputs, dim=1).item()
            
            # Get class names using reverse mapping
            predicted_class = reverse_class_map[predicted_idx]
            true_class = reverse_class_map[true_label.item()]

            correct_prediction = predicted_class == true_class
            correctness_text = "pareizi" if correct_prediction else "nepareizi"
            print(f"{idx + 1}. datu rindiņā klasifikatora rezultāts: {predicted_class}; reālā suga: {true_class}, {correctness_text}")

            if correct_prediction:
                correct += 1

    accuracy = (correct / total) * 100
    print(f"Precizitāte: {accuracy:.0f}% ({correct}/{total})")
    
    return accuracy
    

# Definē funkciju test, kuru izsaucot, modelis tiek pārbaudīts ar validācijas vai testa datu kopu
def test():
    if TRAIN == False:
        # Download model weights from URLs if in testing mode
        # For the normal model
        download_weights_from_url(MODEL_URL, "iris_model.pth")
        # For the minimal model
        download_weights_from_url(MINIMAL_MODEL_URL, "minimal_iris_model.pth")
        
    print("Testing normal model...")
    testModel(IrisModel, "iris_model.pth")
    print("Testing minimal model...")
    testModel(MinimalLinearIrisModel, "minimal_iris_model.pth")

def download_weights_from_url(url, save_path):
    print(f"Downloading model weights from {url}...")
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for HTTP errors
        
        # Save the downloaded content to a file
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f"Successfully downloaded weights to {save_path}")
        
    except Exception as e:
        print(f"Error downloading weights: {e}")
        return False
    
    return True
    

# Definē funkciju saveWeights, kuru izsaucot, tiek saglabāti modeļa svari
def saveWeights():
    print("Saving weights...")
    pass
    # --- Jūsu kods šeit ---

# Definē funkciju loadWeights, kuru izsaucot, no URL MODEL_URL tiek ielādēti modeļa svari un piešķirti mūsu modelim
def loadWeights():
    print(f"Loading weights from {MODEL_URL}...")
    pass
    # --- Jūsu kods šeit ---

### Datu ielādes bloks
Šajā koda blokā ielādē nepieciešamo datu kopa un sagatavo darbam ar modeli

In [4]:
if TRAIN == True:
    print("Ielādē treniņa datu kopu")
    # NB! Kaggle Settings jābūt 'Turn On Internet', savādāk no tīkla neko nevarēs ielādēt, būs kļūda 'Temporary failure in name resolution' 
    iris_dataframe = pd.read_csv(TRAIN_DATA_URL)
    
else:
    print("Ielādē testa datu kopu")
    iris_dataframe = pd.read_csv(TEST_DATA_URL)

iris_dataframe

Ielādē testa datu kopu


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,4.4,3.2,1.3,0.2,Iris-setosa
1,5.0,3.5,1.6,0.6,Iris-setosa
2,5.1,3.8,1.9,0.4,Iris-setosa
3,4.8,3.0,1.4,0.3,Iris-setosa
4,5.8,2.6,4.0,1.2,Iris-versicolor
5,5.0,2.3,3.3,1.0,Iris-versicolor
6,5.6,2.7,4.2,1.3,Iris-versicolor
7,5.7,3.0,4.2,1.2,Iris-versicolor
8,5.8,2.7,5.1,1.9,Iris-virginica
9,6.8,3.2,5.9,2.3,Iris-virginica


### Modeļa apmācīšanas bloks
Šis bloks tiek palaists tikai tad, ja TRAIN == True.  
Satur tikai funkciju train.

In [5]:
if TRAIN == True:
    train()

### Modeļa saglabāšanas bloks
Šis bloks tiek palaists tikai tad, ja TRAIN == True.  
Satur tikai funkciju saveWeights.

In [6]:
if TRAIN == True:
    saveWeights()

### Modeļa ielādēšanas bloks
Šis bloks tiek palaists tikai tad, ja TRAIN == False.  
Satur tikai funkciju loadWeights.

In [7]:
if TRAIN == False:
    loadWeights()

Loading weights from https://dl.dropbox.com/scl/fi/r6iy31winl7u79bacak93/iris_model-1.pth?rlkey=ca646luvn4mne1cbvp2osdqpa&st=dx1to0em&dl=1...


### Modeļa pārbaudes kods
Šis bloks tiek palaists jebkurā gadījumā. Visiem iepriekšējiem blokiem jānostrādā pareizi atkarībā no TRAIN stāvokļa, lai šis kods paņemtu no jauna uztrenētu modeli un pārbaudīto to uz validācijas datu kopas, vai ielādētu saglabātu modeli un pārbaudītu to uz testa datu kopas.

Testa palaišanai jāizdrukā:
1. visu pārbaužu rezultāti viegli saprotamā formā (12 rindiņas, pa vienai katram validācijas kopas piemēram)
2. kopējais precizitātes novērtējums.
   
Paraugs iespējamam adekvātam noformējumam:

- 1\. datu rindiņā klasifikatora rezultāts: Iris-setosa; reālā suga: Iris-setosa, pareizi
- 2\. datu rindiņā klasifikatora rezultāts: Iris-versicolor; reālā suga: Iris-setosa, nepareizi
- ...
- 12\. datu rindiņā klasifikatora rezultāts: Iris-virginica; reālā suga: Iris-virginica, pareizi
    Precizitāte: 58% (7/12)


In [8]:
test()

Successfully downloaded weights to iris_model.pth
Successfully downloaded weights to minimal_iris_model.pth
Testing normal model...
Testing IrisModel model...
1. datu rindiņā klasifikatora rezultāts: Iris-setosa; reālā suga: Iris-setosa, pareizi
2. datu rindiņā klasifikatora rezultāts: Iris-setosa; reālā suga: Iris-setosa, pareizi
3. datu rindiņā klasifikatora rezultāts: Iris-setosa; reālā suga: Iris-setosa, pareizi
4. datu rindiņā klasifikatora rezultāts: Iris-setosa; reālā suga: Iris-setosa, pareizi
5. datu rindiņā klasifikatora rezultāts: Iris-versicolor; reālā suga: Iris-versicolor, pareizi
6. datu rindiņā klasifikatora rezultāts: Iris-versicolor; reālā suga: Iris-versicolor, pareizi
7. datu rindiņā klasifikatora rezultāts: Iris-versicolor; reālā suga: Iris-versicolor, pareizi
8. datu rindiņā klasifikatora rezultāts: Iris-versicolor; reālā suga: Iris-versicolor, pareizi
9. datu rindiņā klasifikatora rezultāts: Iris-virginica; reālā suga: Iris-virginica, pareizi
10. datu rindiņā kla